In [131]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import os
import seaborn as sns
from matplotlib import rcParams
import warnings
warnings.filterwarnings("ignore")

In [132]:
# Reading weather data
final_weather=pd.read_csv("cleaned_data/weather_data.csv")


In [133]:
#seting county and state as index
df_weather=final_weather.set_index(["County", "State"])
#only lat and lon colums
df_weather=df_weather[["Lon", "Lat", "Max_temp","Min_temp"]]

## Field Crops

In [134]:
#reading all field crops csv for different states data to pandas dataframe
field_crops_data=[]
for i in range(1,8):
    df=pd.read_csv(f'data/field_crops{i}.csv')
    field_crops_data.append(df)
#concating all states datas to one dataframe
field_crops=pd.concat([data for data in field_crops_data], axis=0)

In [135]:
#all columns of dataframe
field_crops.columns

Index(['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State',
       'State ANSI', 'Ag District', 'Ag District Code', 'County',
       'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed',
       'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value',
       'CV (%)'],
      dtype='object')

In [136]:
#selecting only important columns
field_crops=field_crops[['County','State','Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value']]

In [137]:
#replacing string (D) with 0 in value columns and converting to float
field_crops['Value']=field_crops['Value'].replace({' (D)':0, ' (Z)':0})
field_crops['Value']=field_crops['Value'].str.replace(',', '').astype(float)

In [138]:
#All field crops names
field_crops_list=field_crops['Commodity'].unique().tolist()


In [139]:
#value for each comodity with county
field_crops_df=field_crops.groupby(['County', 'State', 'Commodity']).agg({'Value':sum})
field_crops_df.reset_index(inplace=True)
field_crops_df

,County,State,Commodity,Value
0,ABBEVILLE,SOUTH CAROLINA,CORN,579.0
1,ABBEVILLE,SOUTH CAROLINA,COTTON,4.0
2,ABBEVILLE,SOUTH CAROLINA,"FIELD CROPS, OTHER",825137.0
3,ABBEVILLE,SOUTH CAROLINA,GRAIN,10021.0
4,ABBEVILLE,SOUTH CAROLINA,HAY,31587.0
...,...,...,...,...
34678,ZIEBACH,SOUTH DAKOTA,PEAS,27952.0
34679,ZIEBACH,SOUTH DAKOTA,SORGHUM,6.0
34680,ZIEBACH,SOUTH DAKOTA,SOYBEANS,341278.0
34681,ZIEBACH,SOUTH DAKOTA,SUNFLOWER,93924215.0


In [140]:
#making state , county captilize
field_crops_df['State']=field_crops_df['State'].str.capitalize()
field_crops_df['County']=field_crops_df['County'].str.capitalize()


In [141]:
def state_abbr(x):
    return (x.replace( "Alaska", "AK").replace("Alabama", "AL").replace("Arizona", "AZ").replace("Arkansas", "AR").replace("California", "CA").replace("Colorado", "CO").replace("Connecticut", "CT").replace("Delaware", "DE").replace("Florida", "FL").replace("Georgia", "GA").replace("Hawaii", "HI").replace("Idaho", "ID").replace("Illinois", "IL").replace("Indiana", "IN").replace("Iowa", "IA").replace("Kansas", "KS").replace("Kentucky", "KY").replace("Louisiana", "LA").replace("Maine", "ME").replace("Maryland", "MD").replace("Massachusetts", "MA").replace("Michigan", "MI").replace("Minnesota", "MN").replace("Mississippi", "MS").replace("Missouri","MO").replace("Montana", "MT").replace("Nebraska", "NE").replace("Nevada", "NV").replace("New hampshire", "NH").replace("New jersey", "NJ").replace("New mexico", "NM").replace("New york", "NY").replace('North carolina', "NC").replace("North dakota", "ND").replace("Ohio", "OH").replace("Oklahoma", "OK").replace("Oregon", "OR").replace("Pennsylvania", "PA").replace("Rhode island", "RI").replace("South carolina", "SC").replace("South dakota", "SD").replace("Tennessee", "TN").replace("Texas", "TX").replace("Utah", "UT").replace("Vermont", "VT").replace("Virginia", "VA").replace("Washington", "WA").replace("West virginia", "WV").replace("Wisconsin", "WI").replace("Wyoming", "WY"))

In [142]:
field_crops_df['State'] = field_crops_df['State']. apply(state_abbr)
field_crops_df

,County,State,Commodity,Value
0,Abbeville,SC,CORN,579.0
1,Abbeville,SC,COTTON,4.0
2,Abbeville,SC,"FIELD CROPS, OTHER",825137.0
3,Abbeville,SC,GRAIN,10021.0
4,Abbeville,SC,HAY,31587.0
...,...,...,...,...
34678,Ziebach,SD,PEAS,27952.0
34679,Ziebach,SD,SORGHUM,6.0
34680,Ziebach,SD,SOYBEANS,341278.0
34681,Ziebach,SD,SUNFLOWER,93924215.0


In [143]:
#setting index county and state
df_field_crops=field_crops_df.set_index(["County", "State"])


In [145]:
#merging with weather to get lat and lon
field_crops_merged = df_field_crops.merge(df_weather, how='inner', left_index=True, right_index=True)
field_crops_merged

Commodity       Value       Lon      Lat  Max_temp  \
County    State                                                                
Abbeville SC                   CORN       579.0  -82.3785  34.1819      7.51   
          SC                 COTTON         4.0  -82.3785  34.1819      7.51   
          SC     FIELD CROPS, OTHER    825137.0  -82.3785  34.1819      7.51   
          SC                  GRAIN     10021.0  -82.3785  34.1819      7.51   
          SC                    HAY     31587.0  -82.3785  34.1819      7.51   
...                             ...         ...       ...      ...       ...   
Ziebach   SD                   PEAS     27952.0 -101.4999  44.6055     -5.05   
          SD                SORGHUM         6.0 -101.4999  44.6055     -5.05   
          SD               SOYBEANS    341278.0 -101.4999  44.6055     -5.05   
          SD              SUNFLOWER  93924215.0 -101.4999  44.6055     -5.05   
          SD                  WHEAT   7816545.0 -101.4999  44.6055     -5.05   

                 Min_temp  
County    State            
Abbeville SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
...                   ...  
Ziebach   SD        -7.36  
          SD        -7.36  
          SD        -7.36  
          SD        -7.36  
          SD        -7.36  

[31830 rows x 6 columns]

In [146]:
#reseting index
field_crops_merged.reset_index(inplace=True)

In [147]:
#saving combined data as csv 
field_crops_merged.to_csv("cleaned_data/field_crops_merged.csv",encoding="utf-8", index=False)

In [29]:
#saving data as csv 
field_crops_df.to_csv("cleaned_data/field_crops_data.csv",encoding="utf-8", index=False)

In [30]:
field_crops_df.to_json("cleaned_data/field_crops_data.json", orient="index")

## Vegetables

In [161]:
#reading all vegetables csv for different states data to pandas dataframe
vegetables_data=[]
for i in range(8):
    df=pd.read_csv(f'data/vegetables{i}.csv')
    vegetables_data.append(df)
#concating all states datas to one dataframe
vegetables=pd.concat([data for data in vegetables_data], axis=0)

In [162]:
#selecting only important columns
vegetables=vegetables[['County','State','Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value']]

In [163]:
#replacing string (D) with 0 in value columns and converting to float
vegetables['Value']=vegetables['Value'].replace({' (D)':0, ' (Z)':0})
vegetables['Value']=vegetables['Value'].str.replace(',', '').astype(float)

In [164]:
vegetables.head()

,County,State,Commodity,Data Item,Domain,Domain Category,Value
0,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA - ACRES HARVESTED",TOTAL,NOT SPECIFIED,NaN
1,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA - OPERATIONS WITH AREA HARV...",TOTAL,NOT SPECIFIED,1.0
2,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA, FRESH MARKET - ACRES HARVE...",TOTAL,NOT SPECIFIED,NaN
3,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA, FRESH MARKET - OPERATIONS ...",TOTAL,NOT SPECIFIED,1.0
4,AUTAUGA,ALABAMA,BEANS,"BEANS, SNAP - ACRES HARVESTED",TOTAL,NOT SPECIFIED,NaN


In [165]:
#All vegetables names
vegetables_list=vegetables['Commodity'].unique().tolist()


In [166]:
#value  for each comodity with county
vegetables_df=field_crops.groupby(['County', 'State', 'Commodity']).agg({'Value':sum})
vegetables_df.reset_index(inplace=True)
vegetables_df

,County,State,Commodity,Value
0,ABBEVILLE,SOUTH CAROLINA,CORN,579.0
1,ABBEVILLE,SOUTH CAROLINA,COTTON,4.0
2,ABBEVILLE,SOUTH CAROLINA,"FIELD CROPS, OTHER",825137.0
3,ABBEVILLE,SOUTH CAROLINA,GRAIN,10021.0
4,ABBEVILLE,SOUTH CAROLINA,HAY,31587.0
...,...,...,...,...
34678,ZIEBACH,SOUTH DAKOTA,PEAS,27952.0
34679,ZIEBACH,SOUTH DAKOTA,SORGHUM,6.0
34680,ZIEBACH,SOUTH DAKOTA,SOYBEANS,341278.0
34681,ZIEBACH,SOUTH DAKOTA,SUNFLOWER,93924215.0


In [167]:
#making state , county captilize
vegetables_df['State']=vegetables_df['State'].str.capitalize()
vegetables_df['County']=vegetables_df['County'].str.capitalize()


In [168]:
vegetables_df['State'] = vegetables_df['State']. apply(state_abbr)
vegetables_df

,County,State,Commodity,Value
0,Abbeville,SC,CORN,579.0
1,Abbeville,SC,COTTON,4.0
2,Abbeville,SC,"FIELD CROPS, OTHER",825137.0
3,Abbeville,SC,GRAIN,10021.0
4,Abbeville,SC,HAY,31587.0
...,...,...,...,...
34678,Ziebach,SD,PEAS,27952.0
34679,Ziebach,SD,SORGHUM,6.0
34680,Ziebach,SD,SOYBEANS,341278.0
34681,Ziebach,SD,SUNFLOWER,93924215.0


In [169]:
#setting index county and state
df_vegetables=vegetables_df.set_index(["County", "State"])


In [170]:
#merging with weather to get lat and lon
vegetables_merged = df_vegetables.merge(df_weather, how='inner', left_index=True, right_index=True)
vegetables_merged

Commodity       Value       Lon      Lat  Max_temp  \
County    State                                                                
Abbeville SC                   CORN       579.0  -82.3785  34.1819      7.51   
          SC                 COTTON         4.0  -82.3785  34.1819      7.51   
          SC     FIELD CROPS, OTHER    825137.0  -82.3785  34.1819      7.51   
          SC                  GRAIN     10021.0  -82.3785  34.1819      7.51   
          SC                    HAY     31587.0  -82.3785  34.1819      7.51   
...                             ...         ...       ...      ...       ...   
Ziebach   SD                   PEAS     27952.0 -101.4999  44.6055     -5.05   
          SD                SORGHUM         6.0 -101.4999  44.6055     -5.05   
          SD               SOYBEANS    341278.0 -101.4999  44.6055     -5.05   
          SD              SUNFLOWER  93924215.0 -101.4999  44.6055     -5.05   
          SD                  WHEAT   7816545.0 -101.4999  44.6055     -5.05   

                 Min_temp  
County    State            
Abbeville SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
...                   ...  
Ziebach   SD        -7.36  
          SD        -7.36  
          SD        -7.36  
          SD        -7.36  
          SD        -7.36  

[31830 rows x 6 columns]

In [171]:
#reseting index
vegetables_merged.reset_index(inplace=True)

In [172]:
#saving combined data as csv 
vegetables_merged.to_csv("cleaned_data/vegetables_merged.csv",encoding="utf-8", index=False)

In [ ]:
#saving data as csv 
vegetables_df.to_csv("cleaned_data/vegetables_data.csv",encoding="utf-8", index=False)

## Fruits

In [173]:
#reading all fruits csv for different states data to pandas dataframe
fruits_data=[]
for i in range(1,8):
    df=pd.read_csv(f'data/fruits{i}.csv')
    fruits_data.append(df)
#concating all states datas to one dataframe
fruits=pd.concat([data for data in fruits_data], axis=0)

In [174]:
#selecting only important columns
fruits=fruits[['County','State','Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value']]

In [175]:
#replacing string (D) with 0 in value columns and converting to float
fruits['Value']=fruits['Value'].replace({' (D)':0, ' (Z)':0})
fruits['Value']=fruits['Value'].str.replace(',', '').astype(float)

In [176]:
#All fruits names
fruits_list=fruits['Commodity'].unique().tolist()


In [177]:
#value for each comodity with county
fruits_df=field_crops.groupby(['County', 'State', 'Commodity']).agg({'Value':sum})
fruits_df.reset_index(inplace=True)
fruits_df

,County,State,Commodity,Value
0,ABBEVILLE,SOUTH CAROLINA,CORN,579.0
1,ABBEVILLE,SOUTH CAROLINA,COTTON,4.0
2,ABBEVILLE,SOUTH CAROLINA,"FIELD CROPS, OTHER",825137.0
3,ABBEVILLE,SOUTH CAROLINA,GRAIN,10021.0
4,ABBEVILLE,SOUTH CAROLINA,HAY,31587.0
...,...,...,...,...
34678,ZIEBACH,SOUTH DAKOTA,PEAS,27952.0
34679,ZIEBACH,SOUTH DAKOTA,SORGHUM,6.0
34680,ZIEBACH,SOUTH DAKOTA,SOYBEANS,341278.0
34681,ZIEBACH,SOUTH DAKOTA,SUNFLOWER,93924215.0


In [178]:
#making state , county captilize
fruits_df['State']=fruits_df['State'].str.capitalize()
fruits_df['County']=fruits_df['County'].str.capitalize()


In [179]:
fruits_df['State'] = fruits_df['State']. apply(state_abbr)
fruits_df

,County,State,Commodity,Value
0,Abbeville,SC,CORN,579.0
1,Abbeville,SC,COTTON,4.0
2,Abbeville,SC,"FIELD CROPS, OTHER",825137.0
3,Abbeville,SC,GRAIN,10021.0
4,Abbeville,SC,HAY,31587.0
...,...,...,...,...
34678,Ziebach,SD,PEAS,27952.0
34679,Ziebach,SD,SORGHUM,6.0
34680,Ziebach,SD,SOYBEANS,341278.0
34681,Ziebach,SD,SUNFLOWER,93924215.0


In [180]:
#setting index county and state
df_fruits=fruits_df.set_index(["County", "State"])


In [181]:
#merging with weather to get lat and lon
fruits_merged = df_fruits.merge(df_weather, how='inner', left_index=True, right_index=True)

In [182]:
#reseting index
fruits_merged.reset_index(inplace=True)

In [183]:
#saving combined data as csv 
fruits_merged.to_csv("cleaned_data/fruits_merged.csv",encoding="utf-8", index=False)

In [51]:
#saving data as csv 
fruits_df.to_csv("cleaned_data/fruits_data.csv",encoding="utf-8", index=False)

In [ ]:
## All Crops

In [53]:
all_crops=['BARLEY','CORN','COTTON','GRAIN','HAY','HAYLAGE',
 'OATS','PEANUTS','RYE','SOYBEANS','WHEAT','GRASSES',
 'LEGUMES','PEAS','SORGHUM','SESAME','SUNFLOWER','HERBS','CANOLA','BEANS','BUCKWHEAT',
 'MINT','JOJOBA','CHICKPEAS','RICE','CAMELINA','MILLET','MAPLE SYRUP','HOPS','SAFFLOWER',
 'POPCORN','MUSTARD','TRITICALE','WILD RICE','DILL','SUGARBEETS','TARO','LENTILS',
 'FLAXSEED','TOBACCO','RAPESEED','SUGARCANE','SWITCHGRASS','MISCANTHUS','GUAR',
 'APPLES','BERRY TOTALS','BLUEBERRIES','FIGS','GRAPES','NECTARINES','NON-CITRUS','ORCHARDS',
 'PEACHES','PEARS','PECANS','PLUMS','TREE NUT','PERSIMMONS','BLACKBERRIES','CITRUS TOTALS',
 'GRAPEFRUIT','LEMONS','LIMES','POMEGRANATES','RASPBERRIES','TANGERINES','CHESTNUTS','APRICOTS',
 'STRAWBERRIES','ALMONDS','HAZELNUTS','KUMQUATS','NON-CITRUS','ORANGES','CHERRIES','ELDERBERRIES',
 'TANGELOS','WALNUTS','ARONIA BERRIES','BERRIES','PASSION FRUIT','KIWIFRUIT','PLUM-APRICOT HYBRIDS','OLIVES',
 'GUAVAS','CURRANTS','PISTACHIOS','DATES','CITRUS','BOYSENBERRIES','AVOCADOS','MACADAMIAS',
 'LOGANBERRIES','PRUNES','CHERIMOYAS','PAPAYAS','BANANAS','COFFEE','MANGOES','CRANBERRIES','PINEAPPLES',
 'BEANS','CUCUMBERS','EGGPLANT','GREENS','MELONS','OKRA','PEAS','PEPPERS','POTATOES','PUMPKINS','SQUASH','SWEET CORN',
 'SWEET POTATOES','TOMATOES','TURNIPS','VEGETABLES','CABBAGE','ONIONS','BROCCOLI','LETTUCE',
 'BEETS','CARROTS','SPINACH','RADISHES','GARLIC','HERBS','ASPARAGUS','BRUSSELS SPROUTS','ESCAROLE','CAULIFLOWER',
 'GINGER ROOT','HORSERADISH','GINSENG','CELERY','PARSLEY','RHUBARB','DAIKON','ARTICHOKES','WATERCRESS','CHICORY']

In [102]:
all_crops=[ i.capitalize() for i in all_crops]

## Joining tables

In [124]:
final_weather=pd.read_csv("cleaned_data/weather_data.csv")

In [126]:
df1=field_crops_df.set_index(["County", "State"])

In [127]:
final_weather['County']=final_weather['County'].str.split(" ").str[0]

In [128]:
df_weather=final_weather.set_index(["County", "State"])
df_weather=df_weather[["Lon", "Lat", "Max_temp","Min_temp"]]

In [129]:
df_merged = df1.merge(df_weather, how='inner', left_index=True, right_index=True)

In [130]:
df_merged.head(50)

Commodity       Value       Lon      Lat  \
County    State                                                            
Abbeville SC                         CORN       579.0  -82.3785  34.1819   
          SC                       COTTON         4.0  -82.3785  34.1819   
          SC           FIELD CROPS, OTHER    825137.0  -82.3785  34.1819   
          SC                        GRAIN     10021.0  -82.3785  34.1819   
          SC                          HAY     31587.0  -82.3785  34.1819   
          SC                HAY & HAYLAGE     33232.0  -82.3785  34.1819   
          SC                      HAYLAGE      6254.0  -82.3785  34.1819   
          SC                         OATS         4.0  -82.3785  34.1819   
          SC                         PEAS         2.0  -82.3785  34.1819   
          SC                          RYE         1.0  -82.3785  34.1819   
          SC                     SOYBEANS     48289.0  -82.3785  34.1819   
          SC                    SUNFLOWER         9.0  -82.3785  34.1819   
          SC                        WHEAT     73080.0  -82.3785  34.1819   
Acadia    LA           FIELD CROPS, OTHER   5362113.0  -92.3459  30.3446   
          LA                        GRAIN  74644178.0  -92.3459  30.3446   
          LA                          HAY     44434.0  -92.3459  30.3446   
          LA                HAY & HAYLAGE     23151.0  -92.3459  30.3446   
          LA                      HAYLAGE      2462.0  -92.3459  30.3446   
          LA                         RICE  68584457.0  -92.3459  30.3446   
          LA                     SOYBEANS   1267893.0  -92.3459  30.3446   
          LA                    SUGARCANE    189160.0  -92.3459  30.3446   
          LA                        WHEAT         4.0  -92.3459  30.3446   
Accomack  VA                       BARLEY     29205.0  -75.6803  37.7159   
          VA                         CORN  16456875.0  -75.6803  37.7159   
          VA                       COTTON         3.0  -75.6803  37.7159   
          VA           FIELD CROPS, OTHER     72015.0  -75.6803  37.7159   
          VA                        GRAIN  28451070.0  -75.6803  37.7159   
          VA                          HAY      3018.0  -75.6803  37.7159   
          VA                HAY & HAYLAGE      1589.0  -75.6803  37.7159   
          VA                      HAYLAGE        90.0  -75.6803  37.7159   
          VA                         MINT         1.0  -75.6803  37.7159   
          VA                     RAPESEED         2.0  -75.6803  37.7159   
          VA                          RYE         1.0  -75.6803  37.7159   
          VA                      SORGHUM    577360.0  -75.6803  37.7159   
          VA                     SOYBEANS  13732665.0  -75.6803  37.7159   
          VA                        WHEAT   3377715.0  -75.6803  37.7159   
Ada       ID                       BARLEY     72126.0 -116.3620  43.7069   
          ID                        BEANS     19605.0 -116.3620  43.7069   
          ID                    CHICKPEAS         2.0 -116.3620  43.7069   
          ID                         CORN  11066146.0 -116.3620  43.7069   
          ID           FIELD CROPS, OTHER  16013471.0 -116.3620  43.7069   
          ID                        GRAIN  15004109.0 -116.3620  43.7069   
          ID                      GRASSES         8.0 -116.3620  43.7069   
          ID     GRASSES & LEGUMES TOTALS       725.0 -116.3620  43.7069   
          ID     GRASSES & LEGUMES, OTHER         6.0 -116.3620  43.7069   
          ID                          HAY    232476.0 -116.3620  43.7069   
          ID                HAY & HAYLAGE    160789.0 -116.3620  43.7069   
          ID                      HAYLAGE    183656.0 -116.3620  43.7069   
          ID                      LEGUMES    105222.0 -116.3620  43.7069   
          ID                         MINT    464036.0 -116.3620  43.7069   

                 Max_temp  Min_temp  
County    State                      
Abb

In [ ]:
final_weather.to_json("cleaned_data/weather.json", orient="index")